In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from tqdm import tqdm_notebook
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier

tqdm.pandas()

In [3]:
structure = pd.read_csv("kaggle/structure.csv")
targets = pd.read_csv("kaggle/targets.csv")
df = pd.read_csv("kaggle/user_activity.csv")

In [4]:
costed_steps = structure[structure.step_cost > 0]

In [322]:
def f(x):
    passed = x[x.action == "passed"]
    not_passed = x[~(x.step_id.isin(passed.step_id))]
    
    #lol-comprehension mean attempts per day { 
    # TODO: optimize it with only unique step
    avg_attempts_per_day = np.mean([t.time.values.size for t in [x[(x.action == "started_attempt") & (x.step_id == id_)] for id_ in x.step_id.unique()]])
    # }
    
    not_passed = not_passed.drop_duplicates("step_id")
    
    # passed steps last 10 days and skipped steps all course  {
    days = 10*3600*24
    border = passed.time.max() - days
    passed_steps = len([1 for t in passed.time if t >= border])
    skipped_steps = not_passed.time.values.size
    # }
    
    # number of stepik-points at passed and viewed steps {
    passed_balls = passed.step_cost.sum()
    viewed_balls = not_passed[not_passed.action == "viewed"].step_cost.sum()
    # }
    
    # average balls per day all course {
    this_day = x.time.min() // 86400
    last_day = x.time.max() // 86400
    t = np.zeros((last_day - this_day, ))
    i, cur_sum = 0, 0
    for _, step in x.sort_values("time").iterrows():
        if step.time // 86400 >= this_day + 1:
            t[i] = cur_sum
            i += 1
            this_day += 1
            cur_sum = 0
        cur_sum += step.step_cost
    if np.sum(t):
        median_balls_per_day = np.median(t)
    else:
        median_balls_per_day = 0
    # }
        
    
    res = [passed_steps, skipped_steps, avg_attempts_per_day, passed_balls, viewed_balls, median_balls_per_day]
    return res

In [323]:
x = df[df.step_id.isin(costed_steps.step_id)].groupby("user_id").get_group(16857)

In [324]:
f(x)

[1, 2, 4.0, 1, 1, 0]

In [325]:
import line_profiler
lp = line_profiler.LineProfiler()
lp.add_function(f)
lp.runctx('f(x)', locals=locals(), globals=globals())
# lp.print_stats()

In [326]:
X_t = df[df.step_id.isin(costed_steps.step_id)].groupby("user_id").progress_apply(f)
Y_t = targets[targets.user_id.isin(X_t.index)]

100%|██████████| 9426/9426 [03:21<00:00, 46.86it/s]


In [327]:
X_t.values, Y_t.passed.values

(array([[0, 68, 0.0, 0, 51, 134.5], [9, 0, 2.3333333333333335, 9, 0, 1.0],
        [9, 0, 2.6666666666666665, 9, 0, 44.0], ...,
        [2, 1, 3.3333333333333335, 2, 1, 0], [0, 2, 1.0, 0, 2, 0],
        [3, 0, 6.0, 3, 0, 0]], dtype=object), array([0, 0, 0, ..., 0, 0, 0]))

In [356]:
def prepare(df, num_features):
    X = df.groupby("user_id").progress_apply(f)
    return X.index, np.array([np.array(x) for x in X.values]).reshape(-1, num_features)

In [331]:
X = np.array([np.array(x) for x in X_t.values]).reshape(-1, 6)
Y = Y_t.passed.ravel()

In [345]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
classifier.fit(X_train, Y_train)
Y_pred = classifier.predict(X_test)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [346]:
from sklearn.metrics import f1_score
f1_score(Y_test, Y_pred)

0.32758620689655177

In [351]:
from sklearn.linear_model import SGDClassifier
classifier = SGDClassifier()

In [354]:
scores = []
for _ in tqdm_notebook(range(1000)):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
    classifier.fit(X_train, Y_train)
    Y_pred = classifier.predict(X_test)
    if Y_pred.sum() > 0:
        score = f1_score(Y_test, Y_pred)
        if score > best_score1:
            best_score1 = max(best_score, score)
        scores.append(score)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

67
62


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

...

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [355]:
np.median(scores)
np.mean(scores)
print()
np.median(scores1)
np.mean(scores1)

0.29565217391304349

0.29601351182153118

0.33150568546674175

0.27065014412505584

In [357]:
user_ids, X_val = prepare(pd.read_csv("kaggle/user_activity_test.csv"), 6)

100%|██████████| 4256/4256 [04:34<00:00, 15.48it/s]


In [358]:
X_train[5], X_val[5]

(array([ 0.,  2.,  1.,  0.,  0.,  0.]),
 array([ 30.        ,   0.        ,   0.16666667,   3.        ,
          0.        ,   0.        ]))

In [362]:
predicted = classifier.predict(X_val)

In [363]:
res = pd.DataFrame({"user_id": user_ids, "passed": predicted}).set_index("user_id")

In [364]:
res.passed.value_counts()

0    3882
1     373
Name: passed, dtype: int64

In [365]:
res.to_csv("result.csv")